In [7]:
import csv
import requests
from bs4 import BeautifulSoup

In [12]:
def read_csv(filename, separator):
    data = []
    with open(filename, 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile, delimiter=separator)
        for row in reader:
            data.append(row)
    return data

# Usage
filename = 'netflix_director.csv'
separator = '\t'
netflix_data = read_csv(filename, separator)

print(f"Read {len(netflix_data)} rows from the CSV file.")

Read 2633 rows from the CSV file.


In [13]:
def find_missing_directors(data):
    missing_directors = []
    for index, row in enumerate(data):
        if not row['director'] or row['director'].strip() == '':
            missing_directors.append((index, row))
    return missing_directors

# Usage
missing_director_data = find_missing_directors(netflix_data)

print(f"Found {len(missing_director_data)} entries with missing director information.")

Found 2633 entries with missing director information.


In [20]:
import requests
import re
from bs4 import BeautifulSoup


In [33]:
import requests

def get_director_from_tmdb(title, type_, api_key):
    search_url = f"https://api.themoviedb.org/3/search/{'movie' if type_ == 'Movie' else 'tv'}"
    params = {
        'api_key': api_key,
        'query': title,
    }

    response = requests.get(search_url, params=params)
    if response.status_code == 200:
        results = response.json().get('results')
        if results:
            # Assuming the first result is the correct one
            movie_id = results[0]['id']
            details_url = f"https://api.themoviedb.org/3/{'movie' if type_ == 'Movie' else 'tv'}/{movie_id}/credits"
            response = requests.get(details_url, params={'api_key': api_key})
            if response.status_code == 200:
                credits = response.json()
                crew = credits.get('crew')
                if crew:
                    director = next((member['name'] for member in crew if member['job'] == 'Director'), None)
                    return director
    return None

# Usage
api_key = '39ee08c8e5e7a16a7ec6c92d6218202c'

for index, row in missing_director_data:
    director = get_director_from_tmdb(row['title'], row['type'], api_key)
    if director:
        netflix_data[index]['director'] = director
        print(f"Updated director for {row['title']} to {director}")
    else:
        print(f"Could not find director for {row['title']}")

# Proceed to save the updated data back to the CSV


Could not find director for Blood & Water
Could not find director for Jailbirds New Orleans
Updated director for Kota Factory to Pratish Mehta
Updated director for Vendetta: Truth, Lies and The Mafia to Davide Gambino
Could not find director for Crime Stories: India Detectives
Could not find director for Dear White People
Could not find director for Falsa identidad
Could not find director for Jaguar
Could not find director for Resurrection: Ertugrul
Could not find director for Love on the Spectrum
Could not find director for Chicago Party Aunt
Could not find director for Sex Education
Could not find director for Squid Game
Could not find director for Tayo and Little Wizards
Could not find director for Angry Birds
Updated director for Chhota Bheem to Rajiv Chilaka
Could not find director for He-Man and the Masters of the Universe
Could not find director for Castle and Castle
Could not find director for Dharmakshetra
Could not find director for Nailed It
Could not find director for Numbe

In [38]:
import os

def save_csv(filename, data, separator):
    # Ensure the filename has the correct extension
    if not filename.endswith('.csv'):
        filename += '.csv'

    # Use the current working directory if the original file's directory is not found
    current_directory = os.path.dirname('/mnt/data/netflix_director.csv')
    if not os.path.exists(current_directory):  # Check if the directory exists
        current_directory = os.getcwd()  # Use the current working directory if it doesn't
        print(f"Directory '/mnt/data' not found. Saving to current directory: {current_directory}")

    output_path = os.path.join(current_directory, filename)

    with open(output_path, 'w', newline='', encoding='utf-8') as csvfile:
        if data:
            fieldnames = data[0].keys()
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames, delimiter=separator)

            writer.writeheader()
            for row in data:
                writer.writerow(row)

    print(f"Updated data saved to {output_path}")

# Usage
output_filename = 'updated_netflix_director'
save_csv(output_filename, netflix_data, separator)

Directory '/mnt/data' not found. Saving to current directory: /content
Updated data saved to /content/updated_netflix_director.csv


In [22]:
def save_csv(filename, data, separator):
    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        if data:
            fieldnames = data[0].keys()
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames, delimiter=separator)

            writer.writeheader()
            for row in data:
                writer.writerow(row)

    print(f"Updated data saved to {filename}")

# Usage
output_filename = 'updated_netflix_director'
save_csv(output_filename, updated_data, separator)

Updated data saved to updated_netflix_director
